# 发现毒蘑菇的相似特征

**说明:**

将 `mushroom.dat` 放在当前目录下。

In [1]:
from numpy import *

## Apriori 中的辅助函数

In [2]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

## Apriori 算法

In [3]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

## 从频繁项集中挖掘关联规则

In [4]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

## 发现毒蘑菇的相似特征

导入数据集

In [5]:
mushDatSet = [line.split() for line in open("./mushroom.dat").readlines()]

使用 Apriori 算法，计算频繁集项

In [6]:
L, suppData = apriori(mushDatSet, minSupport=0.4)

搜索包含有毒特征 2 的频繁项集（仅 1 个）：

In [7]:
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '2'})
frozenset({'34', '2'})
frozenset({'59', '2'})
frozenset({'2', '63'})
frozenset({'2', '85'})
frozenset({'2', '86'})
frozenset({'2', '90'})
frozenset({'2', '39'})


搜索包含有毒特征 2 的频繁项集（仅 2 个）：

In [8]:
for item in L[2]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '2', '85'})
frozenset({'2', '39', '90'})
frozenset({'2', '39', '86'})
frozenset({'2', '86', '90'})
frozenset({'2', '39', '85'})
frozenset({'2', '90', '85'})
frozenset({'2', '86', '85'})
frozenset({'2', '63', '85'})
frozenset({'59', '2', '90'})
frozenset({'59', '2', '86'})
frozenset({'59', '2', '85'})
frozenset({'34', '59', '2'})
frozenset({'59', '2', '39'})
frozenset({'34', '2', '90'})
frozenset({'34', '2', '86'})
frozenset({'34', '2', '85'})
frozenset({'34', '2', '39'})


发现所有频繁项集的关联规则

In [9]:
rules = generateRules(L, suppData, minConf=0.7)

frozenset({'1'}) --> frozenset({'24'}) conf: 0.8406537282941777
frozenset({'53'}) --> frozenset({'36'}) conf: 0.8124999999999999
frozenset({'56'}) --> frozenset({'36'}) conf: 0.934322033898305
frozenset({'56'}) --> frozenset({'39'}) conf: 0.9088983050847458
frozenset({'56'}) --> frozenset({'34'}) conf: 1.0
frozenset({'56'}) --> frozenset({'85'}) conf: 1.0
frozenset({'56'}) --> frozenset({'86'}) conf: 1.0
frozenset({'56'}) --> frozenset({'90'}) conf: 0.9682203389830508
frozenset({'24'}) --> frozenset({'36'}) conf: 0.7455770850884583
frozenset({'110'}) --> frozenset({'34'}) conf: 0.9762376237623762
frozenset({'110'}) --> frozenset({'36'}) conf: 0.9524752475247525
frozenset({'110'}) --> frozenset({'85'}) conf: 1.0
frozenset({'110'}) --> frozenset({'86'}) conf: 0.9762376237623762
frozenset({'110'}) --> frozenset({'90'}) conf: 0.9782178217821782
frozenset({'24'}) --> frozenset({'34'}) conf: 0.9557708508845831
frozenset({'24'}) --> frozenset({'85'}) conf: 1.0
frozenset({'24'}) --> frozenset(

frozenset({'90'}) --> frozenset({'34', '36'}) conf: 0.8376068376068376
frozenset({'34'}) --> frozenset({'36', '90'}) conf: 0.7925195855446044
frozenset({'36'}) --> frozenset({'34', '86'}) conf: 0.9691720493247211
frozenset({'86'}) --> frozenset({'34', '36'}) conf: 0.8331650681474003
frozenset({'34'}) --> frozenset({'86', '36'}) conf: 0.834217841799343
frozenset({'85'}) --> frozenset({'34', '36'}) conf: 0.8126538650910882
frozenset({'36'}) --> frozenset({'34', '85'}) conf: 0.9691720493247211
frozenset({'34'}) --> frozenset({'36', '85'}) conf: 0.834217841799343
frozenset({'63'}) --> frozenset({'34', '36'}) conf: 0.8087520259319286
frozenset({'23'}) --> frozenset({'86', '36'}) conf: 0.9691943127962086
frozenset({'23'}) --> frozenset({'36', '85'}) conf: 0.9691943127962086
frozenset({'1'}) --> frozenset({'36', '90'}) conf: 0.9438202247191012
frozenset({'1'}) --> frozenset({'86', '36'}) conf: 0.9713993871297243
frozenset({'1'}) --> frozenset({'36', '85'}) conf: 0.9713993871297243
frozenset({

frozenset({'36', '90'}) --> frozenset({'34', '85'}) conf: 0.9702970297029703
frozenset({'34', '85'}) --> frozenset({'36', '90'}) conf: 0.7925195855446044
frozenset({'34', '36'}) --> frozenset({'90', '85'}) conf: 0.9500151469251741
frozenset({'34', '90'}) --> frozenset({'36', '85'}) conf: 0.8596491228070176
frozenset({'85'}) --> frozenset({'34', '36', '90'}) conf: 0.7720334810438207
frozenset({'36'}) --> frozenset({'34', '90', '85'}) conf: 0.920728126834997
frozenset({'90'}) --> frozenset({'34', '36', '85'}) conf: 0.8376068376068376
frozenset({'34'}) --> frozenset({'36', '90', '85'}) conf: 0.7925195855446044
frozenset({'63', '85'}) --> frozenset({'34', '90'}) conf: 0.8719611021069692
frozenset({'63', '90'}) --> frozenset({'34', '85'}) conf: 0.9572953736654805
frozenset({'34', '63'}) --> frozenset({'90', '85'}) conf: 0.9072512647554807
frozenset({'63'}) --> frozenset({'34', '90', '85'}) conf: 0.8719611021069692
frozenset({'93', '85'}) --> frozenset({'34', '36'}) conf: 0.8286290322580645


frozenset({'110'}) --> frozenset({'34', '36', '86', '85'}) conf: 0.9287128712871286
frozenset({'86', '53', '85'}) --> frozenset({'34', '90'}) conf: 1.0
frozenset({'53', '90', '85'}) --> frozenset({'34', '86'}) conf: 1.0
frozenset({'53', '86', '90'}) --> frozenset({'34', '85'}) conf: 1.0
frozenset({'34', '53', '85'}) --> frozenset({'86', '90'}) conf: 1.0
frozenset({'34', '86', '53'}) --> frozenset({'90', '85'}) conf: 1.0
frozenset({'53', '34', '90'}) --> frozenset({'86', '85'}) conf: 1.0
frozenset({'53', '85'}) --> frozenset({'34', '86', '90'}) conf: 1.0
frozenset({'86', '53'}) --> frozenset({'34', '90', '85'}) conf: 1.0
frozenset({'53', '90'}) --> frozenset({'34', '86', '85'}) conf: 1.0
frozenset({'34', '53'}) --> frozenset({'86', '90', '85'}) conf: 1.0
frozenset({'53'}) --> frozenset({'34', '90', '86', '85'}) conf: 1.0
frozenset({'85', '24', '86'}) --> frozenset({'34', '90'}) conf: 0.9252418645558487
frozenset({'24', '90', '85'}) --> frozenset({'34', '86'}) conf: 0.954627949183303
fro

frozenset({'1', '86'}) --> frozenset({'34', '90', '36'}) conf: 0.9457523029682703
frozenset({'1', '90'}) --> frozenset({'34', '86', '36'}) conf: 0.9705882352941176
frozenset({'1', '36'}) --> frozenset({'34', '86', '90'}) conf: 0.971608832807571
frozenset({'1', '34'}) --> frozenset({'90', '86', '36'}) conf: 0.948178553104156
frozenset({'1'}) --> frozenset({'34', '36', '90', '86'}) conf: 0.9438202247191012
frozenset({'1', '90', '85'}) --> frozenset({'34', '36'}) conf: 0.9705882352941176
frozenset({'1', '36', '85'}) --> frozenset({'34', '90'}) conf: 0.971608832807571
frozenset({'1', '90', '36'}) --> frozenset({'34', '85'}) conf: 1.0
frozenset({'1', '34', '85'}) --> frozenset({'90', '36'}) conf: 0.948178553104156
frozenset({'1', '34', '90'}) --> frozenset({'36', '85'}) conf: 0.9705882352941176
frozenset({'1', '34', '36'}) --> frozenset({'90', '85'}) conf: 0.9762282091917591
frozenset({'1', '85'}) --> frozenset({'34', '90', '36'}) conf: 0.9438202247191012
frozenset({'1', '90'}) --> frozense

前件是有毒特征 2 的所有关联规则：

In [10]:
rules2 = []
for i in range(len(rules)):
    if rules[i][0] == frozenset({'2'}):
        rules2.append(rules[i])
rules2

[(frozenset({'2'}), frozenset({'28'}), 0.8098859315589354),
 (frozenset({'2'}), frozenset({'34'}), 0.9543726235741445),
 (frozenset({'2'}), frozenset({'59'}), 0.8650190114068441),
 (frozenset({'2'}), frozenset({'63'}), 0.8079847908745247),
 (frozenset({'2'}), frozenset({'85'}), 1.0),
 (frozenset({'2'}), frozenset({'86'}), 0.9543726235741445),
 (frozenset({'2'}), frozenset({'90'}), 0.8745247148288975),
 (frozenset({'2'}), frozenset({'39'}), 0.9315589353612168),
 (frozenset({'2'}), frozenset({'28', '85'}), 0.8098859315589354),
 (frozenset({'2'}), frozenset({'39', '90'}), 0.8060836501901141),
 (frozenset({'2'}), frozenset({'39', '86'}), 0.8859315589353612),
 (frozenset({'2'}), frozenset({'86', '90'}), 0.8288973384030418),
 (frozenset({'2'}), frozenset({'39', '85'}), 0.9315589353612168),
 (frozenset({'2'}), frozenset({'85', '90'}), 0.8745247148288975),
 (frozenset({'2'}), frozenset({'85', '86'}), 0.9543726235741445),
 (frozenset({'2'}), frozenset({'63', '85'}), 0.8079847908745247),
 (froze